<a href="https://colab.research.google.com/github/Marcello-Bacchi/ML_exam_unipv/blob/main/MLP_and_1D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# titolo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

# Train and validation data preparation


In [ ]:
! git clone https://github.com/Marcello-Bacchi/ML_exam_unipv.git
! curl https://github.com/Marcello-Bacchi/ML_exam_unipv.git

In [ ]:
datafile = '/content/ML_exam_unipv/resonances_dataset.h5'

f = h5py.File(datafile)
print(f.keys())

In [ ]:
features = np.array(f.get('resonances')[:,0:2000])
target = np.array(f.get('resonances')[:,-1])

print(target.shape, features.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.33)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
del features, target

# MLP model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Activation
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN

In [ ]:
input_shape = X_train.shape[1]
dropoutRate = 0.25

inputArray = Input(shape=(input_shape))

######### FIRST LAYER ##########
x = Dense(30, activation='relu')(inputArray)
x = Dropout(dropoutRate)(x)

######### SECOND LAYER ##########
x = Dense(20)(x)
x = Activation('relu')(x)
x = Dropout(dropoutRate)(x)

output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputArray, outputs=output)


model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()

In [ ]:
batch_size = 128
n_epochs = 50

# train
history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose = 2,
                validation_data=(X_val, y_val),
                callbacks = [
                EarlyStopping(monitor='val_loss', patience=10, verbose=1),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1),
                TerminateOnNaN()])

# Results

In [ ]:
# plot training history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.yscale('log')
plt.title('Training History')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

predict_val = model.predict(X_val)

fpr = {}
tpr = {}
auc1 = {}
#label='%s tagger, auc = %.1f%%'%(label,auc1[label]*100.)
#plt.figure()
fpr, tpr, threshold = roc_curve(y_val, predict_val)
auc1 = auc(fpr, tpr)

plt.plot(tpr, fpr)
plt.semilogy()
plt.xlabel("sig. efficiency")
plt.ylabel("bkg. mistag rate")
plt.ylim(0.000001,1)
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

print(auc1)